<a href="https://colab.research.google.com/github/TheDevPro/mapmatchingusinghmm/blob/main/hmm_mapmatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install osmnx geopandas





import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import numpy as np
from math import exp
import time


def load_real_gps_data_from_excel(file_path):
    df = pd.read_excel(file_path)
    df = df.rename(columns={
        'Column1.latitude': 'lat',
        'Column1.longitude': 'lon',
        'Column1.date_of_count': 'timestamp'
    })
    df = df[['lat', 'lon', 'timestamp']].dropna()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['speed'] = np.random.uniform(0, 60, size=len(df))
    return df

def load_map_data():
    place_name = "Chicago, Illinois, USA"
    graph = ox.graph_from_place(place_name, network_type='all')
    roads = ox.graph_to_gdfs(graph, nodes=False, edges=True)
    roads = roads[['geometry', 'highway']]
    return roads


def calculate_emission_probability(distance, sigma=50):
    return exp(- (distance ** 2) / (2 * sigma ** 2))

def match_single_gps_point(gps_point, map_data):
    distances = map_data.geometry.distance(gps_point.geometry)
    nearest_idx = distances.idxmin()
    min_distance = distances[nearest_idx]
    matched_road_type = map_data.loc[nearest_idx, 'highway']

    emission_prob = calculate_emission_probability(min_distance)

    return matched_road_type, min_distance, emission_prob


def real_time_map_matching(gps_data, map_data):
    print("Starting Real-Time Map Matching...\n")


    gps_gdf = gpd.GeoDataFrame(
        gps_data,
        geometry=[Point(xy) for xy in zip(gps_data['lon'], gps_data['lat'])],
        crs='EPSG:4326'
    ).to_crs(epsg=3857)


    map_data = map_data.to_crs(epsg=3857)

    for idx, gps_point in gps_gdf.iterrows():
        matched_road_type, distance_meters, emission_prob = match_single_gps_point(gps_point, map_data)

        print(f"Point {idx+1}:")
        print(f" Location: ({gps_point['lat']}, {gps_point['lon']})")
        print(f" Nearest Road Type: {matched_road_type}")
        print(f" Distance to Road: {distance_meters:.2f} meters")
        print(f" Emission Probability: {emission_prob:.6f}")
        print("-" * 40)

        time.sleep(1)


file_path = "Book2.xlsx"  # your excel file
real_gps_data = load_real_gps_data_from_excel(file_path)
map_data = load_map_data()

real_time_map_matching(real_gps_data, map_data)


Starting Real-Time Map Matching...

Point 1:
 Location: (41.79459, -87.684058)
 Nearest Road Type: secondary
 Distance to Road: 2.75 meters
 Emission Probability: 0.998493
----------------------------------------
Point 2:
 Location: (41.876773, -87.671635)
 Nearest Road Type: footway
 Distance to Road: 0.22 meters
 Emission Probability: 0.999991
----------------------------------------
Point 3:
 Location: (41.891676, -87.627665)
 Nearest Road Type: secondary
 Distance to Road: 0.36 meters
 Emission Probability: 0.999975
----------------------------------------
Point 4:
 Location: (41.836923, -87.721956)
 Nearest Road Type: secondary
 Distance to Road: 1.74 meters
 Emission Probability: 0.999398
----------------------------------------
Point 5:
 Location: (41.976427, -87.652889)
 Nearest Road Type: primary
 Distance to Road: 1.09 meters
 Emission Probability: 0.999761
----------------------------------------
Point 6:
 Location: (41.954748, -87.720647)
 Nearest Road Type: secondary
 Dist